In [3]:
import numpy as np
import pandas as pd
import random

# Define the user preference and destination feature vectors

# Naziv mesta,Cena smestaja za 1 osobu za 1 nocenje (€),Udaljenost od centra (m),Mogući broj putnika po sobi,Cena prevoza po osobi (€),Tip destinacije
data = pd.read_csv('jovanrankovic02/AIS-Travel-Recommender-System/TravelDestinations.csv')

#user_preferences = np.array([4, 3, 5, 2, 4])
#destination_pool = [
    #np.array([5, 3, 4, 2, 4]),
    #np.array([2, 5, 3, 4, 5]),
   # np.array([4, 4, 5, 1, 3]),
  #  np.array([3, 3, 4, 5, 4]),
 #   np.array([5, 4, 4, 3, 4])
#]
data

FileNotFoundError: [Errno 2] No such file or directory: 'TravelDestinations.csv'

In [ ]:
# Define hyperparameters
num_clones = 5
num_iterations = 10
mutation_rate = 0.1

In [ ]:
# Function to calculate affinity (similarity)
def calculate_affinity(preference, destination):
    return np.linalg.norm(preference - destination)

In [ ]:
# Function to generate clones
def generate_clones(antibodies, num_clones):
    clones = []
    for antibody in antibodies:
        for _ in range(num_clones):
            clone = antibody.copy()
            clones.append(clone)
    return clones

In [ ]:
# Function to apply mutation
def apply_mutation(clones, mutation_rate):
    mutated_clones = []
    for clone in clones:
        if random.random() < mutation_rate:
            mutation_vector = np.random.normal(0, 1, clone.shape)
            clone = clone + mutation_vector
            clone = np.clip(clone, 1, 5)  # Assuming the feature range is 1 to 5
        mutated_clones.append(clone)
    return mutated_clones

In [ ]:
# Main AIS-based recommendation function
def recommend_travel_destination(user_preferences, destination_pool, num_clones, num_iterations, mutation_rate):
    memory_cells = []

    for iteration in range(num_iterations):
        # Calculate affinities
        affinities = [calculate_affinity(user_preferences, destination) for destination in destination_pool]

        # Select the best matching destinations
        selected_indices = np.argsort(affinities)[:num_clones]
        selected_destinations = [destination_pool[i] for i in selected_indices]

        # Generate clones
        clones = generate_clones(selected_destinations, num_clones)

        # Apply mutation
        mutated_clones = apply_mutation(clones, mutation_rate)

        # Evaluate the mutated clones
        clone_affinities = [calculate_affinity(user_preferences, clone) for clone in mutated_clones]

        # Select the best clone
        best_clone_index = np.argmin(clone_affinities)
        best_clone = mutated_clones[best_clone_index]

        # Update memory cells
        memory_cells.append(best_clone)
        
        # Select the best memory cell as the final recommendation
        memory_affinities = [calculate_affinity(user_preferences, cell) for cell in memory_cells]
        best_memory_index = np.argmin(memory_affinities)
        best_memory_cell = memory_cells[best_memory_index]

    return best_memory_cell

In [ ]:
# Get the recommended travel destination
recommended_destination = recommend_travel_destination(user_preferences, destination_pool, num_clones, num_iterations, mutation_rate)
print("Recommended Travel Destination:", recommended_destination)